In [13]:
import numpy as np
import random
import itertools
from collections import defaultdict
import math
import os
from itertools import permutations
import time
from scipy.stats import mode
import re
import string
from unidecode import unidecode
import matplotlib.pyplot as plt
from copy import deepcopy
from tqdm import tqdm

# P4

## Z1

```pl
np(L,P,R) ==> adj(L,P,R), np(L,P,R).
np(L,P,R) ==> np(L,P,R), adj(L,P,R).
```

## Z3

```pl
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Dictionary
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


:- [skladnicaTagsBases].

hasTag(Word, Tag) :- tagAndBase(Word,_Base,Tag).

hasTag(w, prep:loc).
 
:- op(1050, xfx, ==>).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% GRAMMAR
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


likeAdj(adj:L:P:R:_, L, P, R).
likeAdj(ppas:L:P:R:_, L, P, R).
likeAdj(pact:L:P:R:_, L, P, R).
likeAdj(padj:L:P:R:_, L, P, R).

np(L,P,R) ==> adj(L,P,R), np(L,P,R).
np(L,P,R) ==> np(L,P,R), adj(L,P,R).
np(L,P,R) ==> num(L,P,_), np(L,P,R).
np(L,P,R) ==> np(L,P,R), np(_,gen,_).
np(pl,P,R1) ==> np(_,P,R1), [i], np(_,P,_R2).
np(L,P,R) ==> subst(L,P,R).
np(L,P,R) ==> psupst(L,P,R), np(L,P,R).
np(L,P,R) ==> np(L,P,R), [,], np(L,P,R).
np(L,P,R) ==> np(L,P,R), [z], np(L,P,R).
np(L,P,R) ==> np(L,P,R), [do], np(_,_,_).
np(L,P,R) ==> np(L,P,R), [w], np(_,loc,_).
np(L,P,R) ==> np(L,P,R), [we], np(_,loc,_).
np(L,P,R) ==> np(L,P,R), [na], np(_,_,_).
np(L,P,R) ==> np(L,P,R), [z], np(_,_,_).
np(L,P,R) ==> np(_,_,_), [z], np(L,P,R).
np(L,P,R) ==> ger(_,P,R), np(L,P,R).

subst(L,P,R) ==> [X], {hasTag(X,subst:L:P:R)}.
subst(sg,nom,m2) ==> subst(sg,nom,m2), subst(sg,nom,m2).
subst(sg,nom,m1) ==> subst(sg,nom,m1), subst(sg,nom,m1).
subst(L,nom,R) ==> subst(L,nom,R), [w], subst(L,loc,_).

adja() ==> [X], {hasTag(X,adja)}.
adj(L,P,R) ==> [X], {hasTag(X, Tag), likeAdj(Tag,L,P,R)}.
adj(L,P,R) ==> adja(), [-], adj(L,P,R).
adj(L,P,R) ==> adj(L,P,R), [i], adj(L,P,R).

num(L,P,R) ==> [X], {hasTag(X, num:L:P:R:_)}.
num(L,P,R) ==> num(L,P,R), [tys], [.].
num(L,P,R) ==> num(L,P,R), [proc], [.].
num(L,P,R) ==> num(L,P,R), [mln].
num(L,P,R) ==> [ok], [.], num(L,P,R).

psupst(L,P,R) ==> [X], {hasTag(X, psubst:L:P:R)}.

ppron3(L,P,R) ==> [X], {hasTag(X, ppron3:L:P:R:_)}.

ger(L,P,R) ==> [X], {hasTag(X, ger:L:P:R:_:_)}.

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Parse
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
commasToList((X,Y), [X|Rest]) :- 
   !, commasToList(Y,Rest).
commasToList(X,[X]).   


allign( [[W]| Rest], [W|T], Alligment) :-
   !,allign(Rest, T, Alligment). 
allign( [At|Rest], Ts, [ (At,Pref) | ARest]):-
   Pref = [_|_],
   append(Pref, RestT, Ts),
   allign(Rest, RestT, ARest).
allign( [{C}], [], []) :- C.
allign( [], [], []).


   
parse(A,TokensToParse) :-
   (A ==> Right),
   commasToList(Right, ListRight),
   allign(ListRight, TokensToParse, Alligment),
   parsePairs(Alligment).
   
parsePairs([]).
parsePairs([(A,L)| Rest]):-
   parse(A,L),
   parsePairs(Rest).

writeList([A]) :- write(A),!.
writeList([A|As]):- write(A), write(' '),writeList(As).
   
parse0 :-
   see('phrases.pl'),
   %see('bad_phrases3.pl'),
   parsing,
   seen.

parsing :-
   repeat,
   read(L),
   analyze(L),
   L = end_of_file,!.

analyze(end_of_file) :-
   write('DONE!'), nl, !.
analyze(L) :-   
   length(L,N),
   N < 9,
   parse(np(_,_,_), L),
   write('GOOD:'),
   writeList(L),nl,!.
analyze(L) :-
   write('BAD:'), writeList(L),nl,!.


:- parse0.
```

```
C:\Studia\7 semestr\NLP\P4\P4a>swipl -c np_original.pl | find /C "GOOD"                                  % Disabled autoloading (loaded 27 files)                                                                % Disabled autoloading (loaded 2 files)                                                                 % Disabled autoloading (loaded 0 files)                                                                  10213       
```

```
C:\Studia\7 semestr\NLP\P4\P4a>swipl -c np_original.pl | find "GOOD"                                    GOOD:posterunkowy józef pieróg                                                                       GOOD:pan porucznik barański pana                                                                        % Disabled autoloading (loaded 27 files)                                                                % Disabled autoloading (loaded 2 files)                                                                  % Disabled autoloading (loaded 0 files) 
```

## Z4  (7p, E) 
W zadaniu będziemy ponownie losować wersy Pana Tadeusza, ale tym razem nie korzystając z oryginału, lecz tylko z korpusu PolEvala oraz z zanurzeń wektorowych. Dla ułatwienia przygotowany został zbiór poprawnych rytmicznie (Mają one 26 sylab z przerwami po 7, 13 i 20 sylabie, na końcu każdego wersu i przed średniówkami nie ma słów) zdań z PolEvala (zawierających tylko jednosylabowych słowa z pliku supertags). Większość z nich się nie rymuje (ale dla ułatwienie w pliku z tymi zdaniami
zawarte są tylko takie, które da się zrymować, z zachowaniem liczby sylab i tagów gramatycznych
ostatnich słów).

a) Napisz program losujący dwuwers i modyfikujący ostatnie wyrazy (być może nie oba) w wersach
w ten sposób, by się rymowały (z zachowaniem tagu i liczby sylab). W wyborze powinieneś premiować sytuację, w których wyrazy po zrymowaniu są podobne do wyrazów oryginalnych (czyli
wektory ich form bazowych mają możliwie duży iloczyn skalarny). Nie dla każdego dwuwersu to
da się zrobić, powinieneś to uwzględniać przy wyborze dwuwersu. Przykładowy wynik działaniu
programu (potencjalnie użyteczny do testów):

ORYGINAŁ: po zjednoczeniu niemiec dotychczas strzeżony; obszar został otwarty
i przebudowany.

POEZJA: po zjednoczeniu niemiec dotychczas strzeżony; obszar został otwarty i
podpiwniczony.


b) Dodaj do powyższego programu możliwość zamiany wybranych słów na inne. Zamieniać powinieneś tylko czasowniki, rzeczowniki, przysłówki, imiesłowy i przymiotniki. Oczywiście w zamianie zachowujemy tag i staramy się zachować podobieństwo do oryginału. Nie wolno nam też zepsuć rymu. Przykładowe działanie:

ORYGINAŁ: seria dziecięcych skarpet antypoślizgowych; z motywami mieszkańców
obszarów polarnych.

POEZJA: seria dziecięcych skarpet antypoślizgowych; z motywami mieszkańców obszarów szelfowych.

ZMODYFIKOWANA: seria przedszkolnych skarpet antypoślizgowych; z pejzażami parafian terenów magmowych.

In [2]:
vowels = {'a', 'ą', 'e', 'ę', 'i', 'o', 'u', 'y', 'ó'}

def vowels_split(w):
    res = []
    i,j = -1,0
    while j < len(w):
        if w[j] in vowels:
            if i!= -1:   
                res.append(w[i:j])
            i = j
            if j+1<len(w) and w[j] == 'i' and w[j+1] in vowels:
                j+=1
        j+=1
        
    if i!= -1:
        res.append(w[i:])
        
    return res

In [3]:
def is_rhymes(w1, w2):
    return ''.join(vowels_split(w1)[-2:]) == ''.join(vowels_split(w2)[-2:])

In [4]:
POTENTIAL_RHYMES = []
RHYMES = []

with open('rytmiczne_zdania_z_korpusu.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.split()
        i = line.index('[*]')
        if line[0] == 'POTENCJALNY_RYM:':
            POTENTIAL_RHYMES.append([line[1:i], line[i+1:-1]])
        else:
            RHYMES.append([line[1:i], line[i+1:-1]])

In [5]:
TAGS = {}
TAGS_TO_WORDS = {}
with open('./supertags.txt','r',encoding='utf-8') as f:
    for line in f:
        w,t = line[:-1].split(' ')
        TAGS[w] = t
        
        if t in TAGS_TO_WORDS:
            TAGS_TO_WORDS[t].append(w)
        else:
            TAGS_TO_WORDS[t] = [w]

In [6]:
EMBEDDINGS = {}

with open('poleval_base_vectors.txt', 'r', encoding='utf-8') as f:
    c = -1
    for line in f:
        c+=1
        if not c:
            continue

        line = line.split()
        EMBEDDINGS[line[0]] = np.array(list(map(float, line[1:])))

In [7]:
BASE = {}
with open('superbazy.txt', 'r', encoding='utf-8') as f:
    for line in f:
        w1,w2 = line.split()
        BASE[w1]=w2

In [8]:
def get_embedding(w):
    if w in EMBEDDINGS:
        return EMBEDDINGS[w]
    base_w = BASE[w.lower()]
    if base_w in EMBEDDINGS:
        return EMBEDDINGS[base_w]
    return np.zeros(200)

### a)
Napisz program losujący dwuwers i modyfikujący ostatnie wyrazy (być może nie oba) w wersach w ten sposób, by się rymowały (z zachowaniem tagu i liczby sylab). W wyborze powinieneś premiować sytuację, w których wyrazy po zrymowaniu są podobne do wyrazów oryginalnych (czyli wektory ich form bazowych mają możliwie duży iloczyn skalarny). Nie dla każdego dwuwersu to da się zrobić, powinieneś to uwzględniać przy wyborze dwuwersu. Przykładowy wynik działaniu programu (potencjalnie użyteczny do testów):

ORYGINAŁ: po zjednoczeniu niemiec dotychczas strzeżony; obszar został otwarty i przebudowany.

POEZJA: po zjednoczeniu niemiec dotychczas strzeżony; obszar został otwarty i podpiwniczony.

In [47]:
def gen_verses(print_result=False, K=10000, verses=None):
    succeed = False
    c = 0
    while not succeed:
        c+=1
        if verses is None:
            verses_index = np.random.randint(0,len(POTENTIAL_RHYMES))
            verses = POTENTIAL_RHYMES[verses_index]
        
        verses = deepcopy(verses)
        verses_ = deepcopy(verses)
        w1, w2 = verses[0][-1], verses[1][-1]
        t1, t2 = TAGS[w1], TAGS[w2]
        wl1, wl2 = TAGS_TO_WORDS[t1], TAGS_TO_WORDS[t2]
        e1, e2 = get_embedding(w1), get_embedding(w2)
        r1, r2 = ''.join(vowels_split(w1)[-2:]), ''.join(vowels_split(w2)[-2:])
        vc1, vc2 = len(vowels_split(w1)), len(vowels_split(w2))
        wl1 = [(''.join(vowels_split(w)[-2:]), get_embedding(w), w) for w in wl1 if len(vowels_split(w))==vc1]
        wl2 = [(''.join(vowels_split(w)[-2:]), get_embedding(w), w) for w in wl2 if len(vowels_split(w))==vc2]
        
        filtered_options = []
        for o1 in wl1:
            if o1[0] == r2:
                score = o1[1] @ e1 + e2 @ e2
                filtered_options.append((score,o1[2],w2))
        for o2 in wl2:
            if r1 == o2[0]:
                score = e1 @ e1 + o2[1] @ e2
                filtered_options.append((score,w1,o2[2]))
        
        #print(filtered_options)
        if filtered_options:
            succeed = True
            s, cw1, cw2 = max(filtered_options, key=lambda e: e[0])
            verses[0][-1], verses[1][-1] = cw1, cw2
            break
            
        if len(wl1) > K:
            np.random.shuffle(wl1)
            wl1 = wl1[:K]
            wl1.append(w1)
        if len(wl2) > K:
            np.random.shuffle(wl2)
            wl2 = wl2[:K]
            wl2.append(w2)

        
        options = itertools.product(wl1, wl2)
        for w1,w2 in options:
            if w1[0] == w2[0]:
                score = w1[1] @ e1 + w1[1] @ e2
                filtered_options.append((score,w1[2],w2[2]))
        
        if filtered_options:
            succeed = True
            s, cw1, cw2 = max(filtered_options, key=lambda e: e[0])
            verses[0][-1], verses[1][-1] = cw1, cw2
    
    if verses[1][0] == ',':
        verses[0].append(',')
        verses[1] = verses[1][1:]
    
    if print_result:
        print('ORYGINAŁ:')
        print(' '.join(verses_[0]), end='')
        if verses[1][0] == ',':
            print(',')
            print(' '.join(verses_[1][1:]))
        else:
            print('')
            print(' '.join(verses_[1]))
                
        print('\nPOEZJA:')
        print(' '.join(verses[0]), end='')
        if verses[1][0] == ',':
            print(',')
            print(' '.join(verses[1][1:]))
        else:
            print('')
            print(' '.join(verses[1]))
    
    return verses

In [96]:
v = gen_verses()

bo mile przyjąć raczył swej sługi pokorę ,
łaskawym okiem wejrzał na dawida eleonorę


In [26]:
v = gen_verses()

nie ma takiego trybu , panie pośle , aniżeli
projekt w trzecim czytaniu zwracać do brukseli


In [52]:
v = gen_verses(print_result=True)

ORYGINAŁ:
po zakończeniu wojny wrócił do londynu
i rozpoczął karierę aktorską w teatrze

POEZJA:
po zakończeniu wojny wrócił do folkloru
i rozpoczął karierę aktorską w minoru


In [53]:
t = [['po', 'zjednoczeniu', 'niemiec', 'dotychczas', 'strzeżony'],
 ['obszar', 'został', 'otwarty', 'i', 'przebudowany']]

In [54]:
v = gen_verses(verses=t, print_result=True)

ORYGINAŁ:
po zjednoczeniu niemiec dotychczas strzeżony
obszar został otwarty i przebudowany

POEZJA:
po zjednoczeniu niemiec dotychczas strzeżony
obszar został otwarty i podpiwniczony


### b)
Dodaj do powyższego programu możliwość zamiany wybranych słów na inne. Zamieniać powinieneś tylko czasowniki, rzeczowniki, przysłówki, imiesłowy i przymiotniki. Oczywiście w zamianie zachowujemy tag i staramy się zachować podobieństwo do oryginału. Nie wolno nam też zepsuć rymu. Przykładowe działanie:

ORYGINAŁ: seria dziecięcych skarpet antypoślizgowych; z motywami mieszkańców obszarów polarnych.

POEZJA: seria dziecięcych skarpet antypoślizgowych; z motywami mieszkańców obszarów szelfowych.

ZMODYFIKOWANA: seria przedszkolnych skarpet antypoślizgowych; z pejzażami parafian terenów magmowych.

In [10]:
WORDS = set(BASE.keys())
POLIMORF = {}

with open('polimorfologik-2.1.txt', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):        
        base,word,infos = line[:-1].split(';')
        if word in WORDS:
            POLIMORF[word] = infos

In [20]:
TO_EXCHANGE = set()
acceptable = set(['verb', 'subs', 'adv:', 'adj:', 'pact', 'pant', 'pcon', 'ppas'])
for k,v in POLIMORF.items():
    for info in v.split('+'):
        if info[:4] in acceptable:
            TO_EXCHANGE.add(k)
            break

In [39]:
def gen_modified_verses(p=0.5, print_result=False, K=10000, verses=None):
    verses = gen_verses(print_result=print_result, K=K, verses=verses)
    
    lens = [len(verses[0]), len(verses[1])]
    for k in range(2):
        for i in range(lens[k]):
            w = verses[k][i]
            if np.random.random() < p and w in TO_EXCHANGE:
                t = TAGS[w]
                e = get_embedding(w)
                vc = len(vowels_split(w))
                options = TAGS_TO_WORDS[t]
                options = [o for o in options if len(vowels_split(o))==vc]
                if i == lens[k]-1:
                    r = ''.join(vowels_split(w)[-2:])
                    options = [o for o in options if ''.join(vowels_split(o)[-2:])==r]
            
                scored_options = []
                for option in options:
                    if option == w:
                        continue
                    e2 = get_embedding(option)
                    scored_options.append((e2@e, option))
                
                if scored_options:
                    w2 = max(scored_options, key=lambda e:e[0])[1]
                    verses[k][i] = w2
    
    if print_result:            
        print('\nZMODYFIKOWANA POEZJA:')
        print(' '.join(verses[0]), end='')
        if verses[1][0] == ',':
            print(',')
            print(' '.join(verses[1][1:]))
        else:
            print('')
            print(' '.join(verses[1]))
    
    return verses

In [30]:
v = gen_modified_verses(print_result=True)

POEZJA:
jako mówca i pisarz , podziwiany nawet
przez wrogów , już za życia odniósł wielki bawet

ZMODYFIKOWANA POEZJA:
jako poseł i pisarz , podziwiany nawet
przez zdrajców , już za życia odniósł duży bawet


In [57]:
v = gen_modified_verses(print_result=True)

ORYGINAŁ:
co roku była ona oceniana bardzo
pozytywnie i myślę , że nadal tak będzie

POEZJA:
co roku była ona oceniana wszędzie
pozytywnie i myślę , że nadal tak będzie

ZMODYFIKOWANA POEZJA:
co roku była ona oceniana wszędzie
negatywnie i zadam , że nadal skądś będzie


In [58]:
v = gen_modified_verses(verses=t, print_result=True)

ORYGINAŁ:
po zjednoczeniu niemiec dotychczas strzeżony
obszar został otwarty i przebudowany

POEZJA:
po zjednoczeniu niemiec dotychczas strzeżony
obszar został otwarty i podpiwniczony

ZMODYFIKOWANA POEZJA:
po zjednoczeniu niemiec dotychczas strzeżony
obszar został zamknięty i zaokrąglony


In [60]:
t2 = [['seria', 'dziecięcych', 'skarpet', 'antypoślizgowych'], 
      ['z', 'motywami', 'mieszkańców', 'obszarów', 'polarnych']]

In [61]:
v = gen_modified_verses(verses=t2, print_result=True, p=1.0)

ORYGINAŁ:
seria dziecięcych skarpet antypoślizgowych
z motywami mieszkańców obszarów polarnych

POEZJA:
seria dziecięcych skarpet antypoślizgowych
z motywami mieszkańców obszarów szelfowych

ZMODYFIKOWANA POEZJA:
plansza przedszkolnych bluzek poliuretanowych
z pejzażami parafian terenów magmowych
